In [1]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval, run_roster_solver_output

In [2]:
weekday_ = 'berlin_db=0.50_dt=doublepeak.json'
weekend_ = 'berlin_db=0.50_dt=uniform.json'

instance_file_weekday = f"../instances/{weekday_}"
instance_file_weekend = f"../instances/{weekend_}"

shift_file_weekday = f"../shifts/{weekday_}"
shift_file_weekend = f"../shifts/{weekend_}"

base_file = 'berlin_db=1.00'
model = "flex"
OC = 1.5
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3
max_n_shifts = 3

workforce_dict = {0: 5, 1:5, 2:5, 3:5}

sol = run_roster_solver_output(model, instance_file_weekday, shift_file_weekday, 
                               instance_file_weekend, shift_file_weekend, workforce_dict, 
                               OC, RM, GM, h_min, h_max, max_n_diff)

{0: {'shifts_start': {'0': 0, '1': 2, '2': 4}, 'shifts_end': {'0': 4, '1': 6, '2': 8}}, 1: {'shifts_start': {'0': 2}, 'shifts_end': {'0': 6}}, 2: {'shifts_start': {'0': 0, '1': 2, '2': 4}, 'shifts_end': {'0': 4, '1': 6, '2': 8}}, 3: {'shifts_start': {'0': 2}, 'shifts_end': {'0': 6}}}
{0: {'shifts_start': {'0': 0}, 'shifts_end': {'0': 4}}, 1: {'shifts_start': {'0': 0}, 'shifts_end': {'0': 4}}, 2: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 3: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}}
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-12
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 102180 rows, 115955 columns and 406939 nonzeros
Model fingerprint: 0x28fc715b
Variable types: 99120 continuous, 16835 integer (16835 binary)
Coefficient statistics:
  Matrix range     

In [3]:
starts = []

# self.r[(employee, shift_start, day)]
for region in sol.i.regions:
    for employee in sol.i.employees[region]:
        for day in range(7):
            for shift_start in range(8):
                if (employee, shift_start, day) not in sol.r:
                    continue
                value = sol.r[(employee, shift_start, day)].X
                if value > 0.5:
                    starts.append({'employee': employee, 'shift_start': shift_start, 'shift_end': shift_start+4-1, 'day': day})

starts_df = pd.DataFrame(starts)
starts_df

,employee,shift_start,shift_end,day
0,0,2,5,0
1,0,2,5,1
2,0,2,5,2
3,0,2,5,3
4,0,2,5,4
...,...,...,...,...
115,19,2,5,1
116,19,2,5,2
117,19,2,5,3
118,19,2,5,4


[0, 1, 2, 3, 4, 5, 6, 7]

In [42]:
employee_area_assignment = []
for day in sol.i.days:
    for region in sol.i.regions:
        for employee in sol.i.employees[region]:
            for area in sol.i.reg_areas[region]: 
                for theta in sol.i.periods[day]:
                    value = sol.k[(employee, area, theta, day)].X
                    if value > 0.5:
                        employee_area_assignment.append({'employee': employee, 'day': day, 'region': region, 'area': area, 'period': theta})
                   
employee_area_assignment_df = pd.DataFrame(employee_area_assignment).sort_values(['employee', 'day', 'period'])  


In [5]:
employee_area_assignment_df.merge(starts_df, on=['employee', 'day'], how='left').query('period < shift_start')

,employee,day,region,area,period,shift_start,shift_end


In [6]:
employee_area_assignment_df.merge(starts_df, on=['employee', 'day'], how='left').query('period > shift_end')

,employee,day,region,area,period,shift_start,shift_end


In [33]:
starts_df.query('employee == 19')

,employee,shift_start,shift_end,day
114,19,2,5,0
115,19,2,5,1
116,19,2,5,2
117,19,2,5,3
118,19,2,5,4
119,19,4,7,5


In [37]:
employee_area_assignment_df.query('employee == 0')

,employee,day,region,area,period
0,0,0,3,10717,2
1,0,0,3,10585,3
2,0,0,3,10559,4
3,0,0,3,10551,5
81,0,1,3,10559,2
82,0,1,3,10559,3
80,0,1,3,10627,4
83,0,1,3,10555,5
161,0,2,3,10629,2
162,0,2,3,10559,3


In [43]:
employee_area_assignment_df.query('employee == 19')

,employee,day,region,area,period
76,19,0,2,10997,2
78,19,0,2,10999,3
79,19,0,2,10243,4
77,19,0,2,10967,5
159,19,1,2,10999,2
157,19,1,2,10969,3
156,19,1,2,10997,4
158,19,1,2,10967,5
239,19,2,2,10999,2
237,19,2,2,10969,3


In [44]:
employee_summary = (
    employee_area_assignment_df
    .groupby('employee')
    .agg({'day': ['nunique', 'count'], 'region':['nunique', 'unique']})
)
employee_summary.columns = ['n_days', 'periods_worked', 'n_region', 'region']
print(employee_summary.query('n_days > 6'))

Empty DataFrame
Columns: [n_days, periods_worked, n_region, region]
Index: []


In [45]:
employee_summary

,n_days,periods_worked,n_region,region
employee,,,,
0,6,24,1,[3]
1,6,24,1,[3]
2,6,24,1,[3]
3,6,24,1,[3]
4,6,24,1,[3]
5,6,24,1,[1]
6,6,24,1,[1]
7,6,24,1,[1]
8,6,24,1,[1]


In [30]:
employee = 19
day = 5
sum([sol.k[(employee, area, theta, day)].X
    for area in sol.i.reg_areas[region]
    for theta in sol.i.periods[day]]
)

4.0

In [12]:
#employee_area_assignment_df.query('employee == 5')

In [19]:
employee_area_assignment_df.query('employee == 19').groupby('day').agg({'period': 'min'})

,period
day,
0,2
1,2
2,2
3,2
4,2
5,4


In [23]:
for day in range(7):
    print(f'day {day}: {sol.i.shifts[2, day]}')

day 0: [0, 2, 4]
day 1: [0, 2, 4]
day 2: [0, 2, 4]
day 3: [0, 2, 4]
day 4: [0, 2, 4]
day 5: [0, 4]
day 6: [0, 4]


In [15]:
sol.i.shifts_distinct[0]

[0, 2, 4]

In [24]:
u_results = []
for employee in sol.i.employees[2]:
    for shift_start in sol.i.shifts_distinct[0]:
        u_var = sol.U[(employee, shift_start)].X
        u_results.append({'employee': employee, 'shift_start': shift_start, 'value': u_var})

u_results_df = pd.DataFrame(u_results)
u_results_df

,employee,shift_start,value
0,15,0,1.0
1,15,2,1.0
2,15,4,1.0
3,16,0,1.0
4,16,2,1.0
5,16,4,1.0
6,17,0,1.0
7,17,2,1.0
8,17,4,1.0
9,18,0,1.0
